### 獲取財報資訊

這個單元將教您如何獲取財報
首先我們要先看一下財報的網址：
http://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2017&SSEASON=3&REPORT_ID=C
在這一串網址中，有幾個重要的元素：

* step: 1 （不知道做啥用的）
* CO_ID: 1101 （股票代號）
* SYEAR: 2017 （年）
* SSEASON: 3 （季）
* REPORT_ID：個別財報(A)  個體財報(B)  合併報表(C)

這邊的report ID，我們必須要以合併報表優先（90%以上的財報都是合併財報），假如沒有合併報表，我們在抓個體財報喔！

In [1]:
import requests
from io import StringIO
import pandas as pd
import os
import subprocess
import sqlite3
import requests

In [61]:
connection = sqlite3.connect('../Database/stock.db')
cursor = connection.cursor()

### Code mapping v1

In [62]:
# [資產負債表, 損益表, 現金流量表]
Title_to_Code = [
{
    '應收帳款淨額': '1170', 

    '流動資產合計': '11XX', 

    '存貨': '130X',
    '存貨合計': '130X',

    '非流動資產合計': '15XX',

    '不動產、廠房及設備': '1600',
    '不動產、廠房及設備合計': '1600',

    '無形資產': '1780', 
    '無形資產合計': '1780',

    '其他非流動資產': '1900',
    '其他非流動資產合計': '1900',

    '資產總計': '1XXX', 
    '資產總額': '1XXX', 

    '流動負債合計': '21XX', 

    '非流動負債合計': '25XX', 

    '其他非流動負債': '2600',
    '其他非流動負債合計': '2600', 
    
    '特別股股本': '3120', 
    
    '股本合計': '3100', 

    '權益總計': '3XXX', 
    '權益總額': '3XXX'
},

{
    '營業收入合計': '4000', 
    '營業成本合計': '5000',
    '營業毛利（毛損）': '5900', 
    '營業利益（損失）': '6900', 
    '繼續營業單位稅前淨利（淨損）': '7900', # 出現在損益表、現金流量表
    '本期淨利（淨損）': '8200' 
},

{
    '折舊費用': 'A20100', 
    '攤銷費用': 'A20200', 
    '營業活動之淨現金流入（流出）': 'AAAA', 
    '取得不動產、廠房及設備': 'B02700'
}]

In [5]:
CodeSet = {
        '1170',
        '11XX',
        '130X',
        '15XX',
        '1600',
        '1780',
        '1900',
        '1XXX',
        '21XX',
        '25XX',
        '2600',
        '3XXX',
        '3100',
        '3120',
        '4000',
        '5000',
        '5900',
        '6900',
        '7900',
        '8200',
        # 'A20100',
        # 'A20200',
        'AAAA',
        'B02700'}
FinCompCodeSet = {
        '1170',
        '1780',
        '1XXX',
        '3100',
        '3XXX',
        '7900',
        '8200',
        'AAAA',
        'B02700'}

### Code mapping v2
* 代號統一
* 特例處理(金融公司)
    * 代號與中文名稱，跟其他公司不同
    * 只有部分欄位


In [91]:
# [資產負債表, 損益表, 現金流量表]
Standard_Code_mapping = [
{
    '1170': '1170', 
    '應收帳款淨額': '1170', 
    '13000': '1170', # 金融公司
    '應收款項－淨額': '1170', # 金融公司
    # '應收款項-淨額': '1170', # 金融公司 # before 2013

    # 暫時當作應收帳款, 1402.TW
    '應收票據淨額': '1170',
    '1150': '1170', 

    '11XX': '11XX', 
    '流動資產合計': '11XX', 
    '流動資產': '11XX',

    '130X': '130X',
    '存貨': '130X',
    '存貨合計': '130X',

    '15XX': '15XX',
    '非流動資產合計': '15XX',

    '1600': '1600',
    '不動產、廠房及設備': '1600',
    '不動產、廠房及設備合計': '1600',
    # '固定資產成本合計': '1600',  # before 2013
    # '固定資產淨額': '1600',  # before 2013

    '1780': '1780', 
    '無形資產': '1780', 
    '無形資產合計': '1780',
    '無形資產-淨額': '1780', # 金融公司
    '無形資產－淨額': '1780', # 金融公司
    '19000': '1780', # 金融公司

    '1900': '1900',
    '其他非流動資產': '1900',
    '其他非流動資產合計': '1900',
    # '其他資產合計': '1900', # before 2013

    '1XXX': '1XXX', # 一般公司
    '資產總計': '1XXX', 
    '資產總額': '1XXX',
    '19999': '1XXX',  # 金融公司

    '21XX': '21XX', 
    '流動負債合計': '21XX', 
    '流動負債': '21XX',

    '25XX': '25XX', 
    '非流動負債合計': '25XX', 

    '2600': '2600',
    '其他非流動負債': '2600',
    '其他非流動負債合計': '2600', 
    # '其他負債合計': '2600',  # before 2013
    
    # Note: 少數公司有的資訊
    '3120': '3120',
    '特別股股本': '3120',
    
    '3100': '3100', # 一般公司
    '股本合計': '3100', # 一般公司 金融公司
    '31100': '3100', # 金融公司

    '3XXX': '3XXX', # 一般公司
    '權益總計': '3XXX', # 一般公司 金融公司
    '權益總額': '3XXX', # 一般公司 金融公司
    '39999': '3XXX', # 金融公司
    '股東權益總計': '3XXX'
},

{
    '4000': '4000', 
    '營業收入合計': '4000', 

    '5000': '5000',
    '營業成本合計': '5000',

    '5900': '5900', 
    '營業毛利（毛損）': '5900', 
    # '營業毛利(毛損)': '5900', # before 2013

    '6900': '6900', 
    '營業利益（損失）': '6900', 
    # '營業淨利(淨損)': '6900', # before 2013

    # 稅前淨利
    # Note: 同時出現在損益表、現金流量表
    '7900': '7900', # 一般公司 金融公司  
    '繼續營業單位稅前淨利（淨損）': '7900', # 一般公司 金融公司 # 出現在損益表、現金流量表 
    '繼續營業單位稅前損益': '7900', # 一般公司 金融公司 # 出現在損益表、現金流量表 
    '64001': '7900', # 金融公司 (2801.TW)
    # '繼續營業單位稅前合併淨利（淨損）': '7900', # 一般公司 金融公司 # 出現在損益表、現金流量表 # before 2013
    # '繼續營業單位稅前淨利(淨損)': '7900', # 一般公司 金融公司 # 出現在損益表、現金流量表 # before 2013

    # 稅後淨利
    '8200': '8200' , # 一般公司 金融公司
    '本期淨利（淨損）': '8200',
    '69000': '8200', # 金融公司
    '64000': '8200', # 金融公司 (2801.TW)
    '本期稅後淨利（淨損）': '8200'
    # '繼續營業單位稅後合併淨利(淨損)': '8200', # before 2013
    # '繼續營業單位淨利(淨損)': '8200', # before 2013

},

{   # 金融公司的現金流量表，代號與名稱與一般公司相同

    # 金融公司 代號 61001 對應的欄位不一致，改用A0010
    'A00010': '7900', # 金融公司

    # '折舊費用': 'A20100', 
    # '攤銷費用': 'A20200', ˋ
    'AAAA': 'AAAA',   # 一般公司 金融公司
    '營業活動之淨現金流入（流出）': 'AAAA',
    # '營業活動之淨現金流入(流出)': 'AAAA', # before 2013
    # '營業活動之淨現金流入': 'AAAA', # before 2013
    
    'B02700': 'B02700', # 一般公司 金融公司
    '取得不動產、廠房及設備': 'B02700', 
    '取得不動產及設備': 'B02700'
    # '購置固定資產': 'B02700', # before 2013
    # '購置資產': 'B02700' # before 2013
}]

In [ ]:
if len(df[df['Code'] == '股本']) == 0:
    # 股本 = 普通股股本
    df.loc[df['Code'] == '普通股股本', 'Code'] = '3100'

In [69]:
# check all the codes are included in Code
for i in range(3):
    for x in Title_to_Code[i].values():
        if x not in Code:
            print(x)

## year 2019 ~

In [91]:
# res = requests.get('https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=2330&SYEAR=2022&SSEASON=1&REPORT_ID=C')
res = requests.get('https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=2330&SYEAR=2021&SSEASON=3&REPORT_ID=C')
# res.encoding = 'big5'
print(res.headers)
print(res.encoding)
# print(res.text)


{'Server': 'TWSE Server', 'Date': '8 Nov 2022 14:11:22 GMT', 'Content-type': 'text/html', 'X-Content-Type-Options': 'nosniff ', 'X-Content-Security-Policy': "frame-ancestors 'self' *.twse.com.tw ", 'Content-Security-Policy': "frame-ancestors 'self' *.twse.com.tw script-src 'self' https://*.twse.com.tw http://www.google-analytics.com/  https://www.google-analytics.com/ https://code.jquery.com/ 'unsafe-inline' 'unsafe-eval';;", 'Referrer-Policy': 'no-referrer', 'Permissions-Policy': 'accelerometer=(), camera=(), geolocation=(), gyroscope=(), magnetometer=(), microphone=(), payment=(), usb=()', 'X-XSS-Protection': '1;mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Accept-ranges': 'bytes', 'Connection': 'close'}
ISO-8859-1


In [423]:
# 將 res.text 用 StringIO 轉成 檔案 再用 pd.read_html 將 html文字檔轉成 dataframe
try:
    # dfs = pd.read_html(StringIO(res.text))
    dfs = pd.read_html(res.text)[0:3]
except ValueError:
    print('Error: data not available...')
    raise 

In [92]:
print(StringIO(res.text).read())

<html>
<head>
<META HTTP-EQUIV='Content-Type' CONTENT='text/html; charset=big5'>
<?xml version="1.0" encoding="UTF-8"?> 
<html xmlns="http://www.w3.org/1999/xhtml"
xmlns:ifrs-full="http://xbrl.ifrs.org/taxonomy/2017-03-09/ifrs-full"
xmlns:iso4217="http://www.xbrl.org/2003/iso4217"
xmlns:link="http://www.xbrl.org/2003/linkbase"
xmlns:xbrldi="http://xbrl.org/2006/xbrldi"
xmlns:xbrldt="http://xbrl.org/2005/xbrldt"
xmlns:xbrli="http://www.xbrl.org/2003/instance"
xmlns:xlink="http://www.w3.org/1999/xlink"
xmlns:ix="http://www.xbrl.org/2013/inlineXBRL"
xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2015-02-26"
xmlns:tifrs-SCF="http://www.xbrl.org/tifrs/scf/2020-06-30"
xmlns:tifrs-ar="http://www.xbrl.org/tifrs/ar/2020-06-30"
xmlns:tifrs-bsci-ci="http://www.xbrl.org/tifrs/bsci/ci/2020-06-30"
xmlns:tifrs-es="http://www.xbrl.org/tifrs/es/2020-06-30"
xmlns:tifrs-notes="http://www.xbrl.org/tifrs/notes/2020-06-30">
<head> 
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8

In [2]:
# CANNOT directly read html url
# url = 'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1210&SYEAR=2022&SSEASON=1&REPORT_ID=C'
# url = 'http://www.ubus.com.tw/Booking/FareInquiry'
# load html tables into pandas DataFrame
# dfs = pd.read_html(url)

In [433]:
# dfs
# dfs[0]  # 資產負債表
# dfs[1]  # 損益表
dfs[2]  # 現金流量表

,Code,Title,Money
0,NaN,營業活動之現金流量－間接法Cash flows from (used in) operati...,NaN
1,A00010,繼續營業單位稅前淨利（淨損） Profit (loss) from continuing o...,478307103
2,A10000,本期稅前淨利（淨損） Profit (loss) before tax,478307103
3,NaN,調整項目 Adjustments,NaN
4,NaN,收益費損項目 Adjustments to reconcile profit (loss),NaN
...,...,...,...
78,DDDD,匯率變動對現金及約當現金之影響Effect of exchange rate changes...,"(3,621,935)"
79,EEEE,本期現金及約當現金增加（減少）數Net increase (decrease) in cas...,193641319
80,E00100,期初現金及約當現金餘額Cash and cash equivalents at beginn...,660170647
81,E00200,期末現金及約當現金餘額Cash and cash equivalents at end of...,853811966


In [250]:
dfs[0].iloc[:, 0:3]
# dfs[1].iloc[:, 0:3]
# dfs[2].iloc[:, 0:3]

資產負債表Balance Sheet                                                     \
               代號Code                               會計項目Accounting Title   
0                 NaN                                           資產Assets   
1                 NaN                                流動資產　Current assets   
2                1100                  現金及約當現金 Cash and cash equivalents   
3                1110  透過損益按公允價值衡量之金融資產－流動 Current financial assets a...   
4                1120  透過其他綜合損益按公允價值衡量之金融資產－流動 Current financial asse...   
..                ...                                                ...   
85               36XX                    非控制權益 Non-controlling interests   
86               3XXX                                  權益總額 Total equity   
87               3X2X               負債及權益總計　Total liabilities and equity   
88               3998  預收股款（權益項下）之約當發行股數　Equivalent issue shares of a...   
89               3999  母公司暨子公司所持有之母公司庫藏股股數（單位：股）　Number of shares in ...   

                        
   2022年6月30日2022/6/30  
0                  NaN  
1                  NaN  
2           1253189197  
3               214215  
4            120954506  
..                 ...  
85            14108933  
86          2510462125  
87          4345941335  
88                   0  
89                   0  

[90 rows x 3 columns]

In [317]:
i = 2
dfs[i]['Money'].values.tolist()

[492829977,
 492829977,
 219975000,
 4351710,
 22878,
 5044997,
 -5219824,
 -240840,
 141801,
 -3983495,
 59370,
 0,
 245151,
 790740,
 1760843,
 -260642,
 222687689,
 -22297,
 -23410483,
 -357375,
 -33677,
 -24335770,
 -3834683,
 -2794688,
 -54788973,
 10739349,
 355928,
 3426753,
 -2523619,
 98394858,
 110393269,
 55604296,
 278291985,
 771121962,
 -60102845,
 711019117,
 -40949130,
 32886262,
 300,
 -80240090,
 1970939,
 1519511,
 -479857668,
 672885,
 -1768173,
 373262,
 -3805347,
 0,
 4322973,
 530906,
 337473,
 -564005897,
 4048589,
 128157572,
 0,
 500000,
 123240,
 -5683,
 -1122427,
 -142617093,
 -871566,
 0,
 -4657300,
 16438934,
 -5734,
 41191519,
 188199005,
 1064990192,
 1253189197,
 1253189197]

In [425]:
for i in range(3):
    dfs[i] = dfs[i].iloc[:, 0:3]
    dfs[i].columns = ['Code', 'Title', 'Money']

In [311]:
i = 2

dfs[i].dropna(subset='Code', inplace = True)


In [312]:
dfs[i] = dfs[i].astype('string')

In [314]:
dfs[i]['Code'] = dfs[i]['Code'].str.replace('\.0','', regex=True)
dfs[i]['Money'] = dfs[i]['Money'].str.replace('(','-', regex=True)
dfs[i]['Money'] = dfs[i]['Money'].str.replace(')','', regex=True)
dfs[i]['Money'] = dfs[i]['Money'].str.replace(',','', regex=True)

In [316]:
dfs[i]['Money'] = pd.to_numeric(dfs[i]['Money'])

In [313]:
dfs[i].dtypes

Code     string
Title    string
Money    string
dtype: object

In [127]:
dfs[1].values.tolist()

[['4000', '營業收入合計\u3000Total operating revenue', '491075873'],
 ['5000', '營業成本合計\u3000Total operating costs', '217872707'],
 ['5900', '營業毛利（毛損）Gross profit (loss) from operations', '273203166'],
 ['5950', '營業毛利（毛損）淨額Gross profit (loss) from operations', '273203166'],
 ['6100', '推銷費用\u3000Selling expenses', '2062623'],
 ['6200', '管理費用\u3000Administrative expenses', '10500027'],
 ['6300', '研究發展費用\u3000Research and development expenses', '36048540'],
 ['6000', '營業費用合計\u3000Total operating expenses', '48611190'],
 ['6500', '其他收益及費損淨額\u3000Net other income (expenses)', '(801,858)'],
 ['6900', '營業利益（損失）Net operating income (loss)', '223790118'],
 ['7101', '銀行存款利息 Interest income from bank deposits', '991993'],
 ['7102',
  '按攤銷後成本衡量之金融資產利息收入 Interest income from financial assets measured at amortised cost',
  '141396'],
 ['7106',
  '透過其他綜合損益按公允價值衡量之金融資產利息收入 Interest income from financial assets measured at fair value through other comprehensive income',
  '540490'],
 ['7100', '利息收入合計 Total in

In [126]:
dfs[0]['Code'].to_list()

['1100',
 '1110',
 '1120',
 '1136',
 '1139',
 '1170',
 '1180',
 '1210',
 '130X',
 '1470',
 '1476',
 '1479',
 '11XX',
 '1517',
 '1535',
 '1550',
 '1600',
 '1755',
 '1780',
 '1840',
 '1900',
 '1920',
 '1990',
 '15XX',
 '1XXX',
 '2100',
 '2120',
 '2126',
 '2170',
 '2180',
 '2200',
 '2201',
 '2206',
 '2207',
 '2213',
 '2216',
 '2230',
 '2300',
 '2320',
 '2399',
 '21XX',
 '2530',
 '2540',
 '2541',
 '2570',
 '2580',
 '2600',
 '2640',
 '2645',
 '2670',
 '25XX',
 '2XXX',
 '3110',
 '3100',
 '3210',
 '3211',
 '3213',
 '3230',
 '3235',
 '3250',
 '3252',
 '3260',
 '3270',
 '3273',
 '3200',
 '3310',
 '3320',
 '3350',
 '3300',
 '3400',
 '3500',
 '31XX',
 '36XX',
 '3XXX',
 '3X2X',
 '3998',
 '3999']

### 2019 ~  檢查缺漏的欄位

In [31]:
print('rows missing:')

for year in range(2019, 2022):
    for season in range(1, 5):
        param = '2885', str(year), str(season)
        url = f'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={param[0]}&SYEAR={param[1]}&SSEASON={param[2]}&REPORT_ID=C'
        
        print(f'    {year}, Q{season}')
        res = requests.get(url)
        res.encoding = 'big5'
        dfs = pd.read_html(StringIO(res.text))[0:3]

        # dfs[0]    資產負債表
        # dfs[1]    損益表
        # dfs[2]    現金流量表

        # save three financial statements
        for i in range(3):
            # drop useless columns
            dfs[i] = dfs[i].iloc[:, 0:3]
            dfs[i].columns = ['Code', 'Title', 'Money']
            dfs[i].dropna(subset='Code', inplace = True)
            
            # all columns into string (for ease of regex processing)
            dfs[i] = dfs[i].astype('string')

            # eliminate ".0" induced by float type
            dfs[i]['Code'] = dfs[i]['Code'].str.replace('\.0','', regex=True)
        
            # eliminate parenthesis and commas
            pat_repl_dict = {'(': '-', ')':'', ',':''}
            for pat, repl in pat_repl_dict.items():
                dfs[i]['Money'] = dfs[i]['Money'].str.replace(pat, repl, regex=True)
 
        tmpCodeSet = set()
        for i in range(0, 3):
            for key, val in Title_to_Code[i].items():
                if len(dfs[i][dfs[i]['Code'] == key]) != 0:
                    tmpCodeSet.add(val)


        # print(CodeSet.difference(tmpCodeSet))     # 一般公司
        print(FinCompCodeSet.difference(tmpCodeSet))    # 金融公司


rows missing:
    2019, Q1
set()
    2019, Q2
set()
    2019, Q3
set()
    2019, Q4
set()
    2020, Q1
set()
    2020, Q2
set()
    2020, Q3
set()
    2020, Q4
set()
    2021, Q1
set()
    2021, Q2
set()
    2021, Q3
set()
    2021, Q4
set()


### Parsing function

In [107]:
def parse_func(param):
    ''' 
        param: (stock id, year, season) (str, str, str)
    '''
    table_name = '_'.join(['t', param[0], param[1], param[2]])
    url = f'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={param[0]}&SYEAR={param[1]}&SSEASON={param[2]}&REPORT_ID=C'
    
    # create tables (sqlite)
    query = 'CREATE TABLE IF NOT EXISTS ' + table_name + ' (\
        Code TEXT PRIMARY KEY, \
        Title TEXT, \
        Money BIGINT \
        );'
    print(query)
    cursor.execute(query)
    cursor.execute(f"DELETE FROM {table_name};")

    # parsing
    print(f'parsing: {url}')
    res = requests.get(url)
    res.encoding = 'big5'
    dfs = pd.read_html(StringIO(res.text))
    # dfs[0]    資產負債表
    # dfs[1]    損益表
    # dfs[2]    現金流量表

    # save three financial statements
    for i in range(3):
        # drop useless columns
        dfs[i] = dfs[i].iloc[:, 0:3]
        dfs[i].columns = ['Code', 'Title', 'Money']

        # drop useless rows (that has blank in "Code")
        dfs[i].dropna(subset='Code', inplace = True)
        
        dfs[i] = dfs[i].astype('string')

        # eliminate ".0"
        dfs[i]['Code'] = dfs[i]['Code'].str.replace('\.0','', regex=True)
        
        # eliminate parenthesis
        dfs[i]['Money'] = dfs[i]['Money'].str.replace('(','', regex=True)
        dfs[i]['Money'] = dfs[i]['Money'].str.replace(')','', regex=True)
        dfs[i]['Money'] = dfs[i]['Money'].str.replace(',','', regex=True)
        dfs[i]['Money'] = pd.to_numeric(dfs[i]['Money'])
        
        # load data into sqlite
        data = dfs[i].values.tolist()
        query = "INSERT INTO " + table_name + " VALUES (?, ?, ?);"
        cursor.executemany(query, data)

        # dfs[i].to_csv(path, encoding='utf-8', index=False)

    connection.commit()

In [108]:
# (stock id, year, season)
query_param = [('2330', '2022', '1')]

for param in query_param:
    parse_func(param)
    

CREATE TABLE IF NOT EXISTS t_2330_2022_1 (        Code TEXT PRIMARY KEY,         Title TEXT,         Money BIGINT         );
parsing: https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=2330&SYEAR=2022&SSEASON=1&REPORT_ID=C


In [45]:
table = "t_2330_2021_2"
result = cursor.execute(f"SELECT * FROM {table}")
for row in result:
    print(row)

('1100', '現金及約當現金 Cash and cash equivalents', 747982082)
('1110', '透過損益按公允價值衡量之金融資產－流動 Current financial assets at fair value through profit or loss', 58325)
('1120', '透過其他綜合損益按公允價值衡量之金融資產－流動 Current financial assets at fair value through other comprehensive income', 122794529)
('1136', '按攤銷後成本衡量之金融資產－流動 Current financial assets at amortised cost', 0)
('1139', '避險之金融資產－流動 Current financial assets for hedging', 310)
('1170', '應收帳款淨額 Accounts receivable, net', 177475493)
('1180', '應收帳款－關係人淨額 Accounts receivable due from related parties, net', 784397)
('1210', '其他應收款－關係人 Other receivables due from related parties', 1711652)
('130X', '存貨 Current inventories', 170438686)
('1470', '其他流動資產 Other current assets', 19858549)
('1476', '其他金融資產－流動 Other current financial assets', 12450123)
('1479', '其他流動資產－其他 Other current assets, others', 7408426)
('11XX', '流動資產合計 Total current assets', 1241104023)
('1517', '透過其他綜合損益按公允價值衡量之金融資產－非流動 Non-current financial assets at fair value through other comprehe

## year 2013 ~ 2018

In [375]:
res = requests.get('https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID=2330&SYEAR=2018&SSEASON=1&REPORT_ID=C')
res.encoding = 'big5'

In [376]:
# 將 res.text 用 StringIO 轉成 檔案 再用 pd.read_html 將 html文字檔轉成 dataframe
dfs = pd.read_html(StringIO(res.text))[1:4]

In [388]:
# dfs
# dfs[0]  # 資產負債表
# dfs[1]  # 損益表
dfs[2]  # 現金流量表

,Code,Money
0,營業活動之現金流量－間接法,NaN
1,繼續營業單位稅前淨利（淨損）,99943621.0
2,本期稅前淨利（淨損）,99943621.0
3,調整項目,NaN
4,收益費損項目,NaN
...,...,...
75,匯率變動對現金及約當現金之影響,-5248235.0
76,本期現金及約當現金增加（減少）數,24391267.0
77,期初現金及約當現金餘額,553391696.0
78,期末現金及約當現金餘額,577782963.0


In [377]:
for i in range(3):
    dfs[i] = dfs[i].iloc[:, 0:2]
    dfs[i].columns = ['Code', 'Money']

In [237]:
# len(dfs[1][dfs[1]['Code'] == '權益總計'])
# dfs[1][dfs[1]['Code'] == '權益總計']
# dfs[1][dfs[1]['Code'] == '3XXX']
# dfs[1][dfs[1]['Code'] == '權益總計']['Code']

In [226]:
dfs[1].loc[dfs[1]['Code'] == '權益總計', 'Code'] = '3XXX'   # use this
# dfs[1][dfs[1]['Code'] == '權益總計']['Code'] = '3XXX' # don't use this

In [192]:
i = 3

# drop useless rows (that has blank in "Money")
dfs[i].dropna(subset='Money', inplace = True)

dfs[i].values.tolist()


[['繼續營業單位稅前淨利（淨損）', 286428171.0],
 ['本期稅前淨利（淨損）', 286428171.0],
 ['折舊費用', 213318950.0],
 ['攤銷費用', 3197428.0],
 ['預期信用減損損失（利益）數／呆帳費用提列（轉列收入）數', -2279.0],
 ['透過損益按公允價值衡量金融資產及負債之淨損失（利益）', 244799.0],
 ['利息費用', 2175318.0],
 ['利息收入', -10543592.0],
 ['股利收入', -158358.0],
 ['採用權益法認列之關聯企業及合資損失（利益）之份額', -1946111.0],
 ['處分及報廢不動產、廠房及設備損失（利益）', 789005.0],
 ['處分無形資產損失（利益）', -436.0],
 ['處分投資損失（利益）', 774784.0],
 ['金融資產減損損失', 0.0],
 ['非金融資產減損損失', 488336.0],
 ['未實現銷貨利益（損失）', 188528.0],
 ['未實現外幣兌換損失（利益）', 1863969.0],
 ['其他項目', 2494.0],
 ['收益費損項目合計', 210392835.0],
 ['持有供交易之金融資產（增加）減少', 639804.0],
 ['應收帳款（增加）減少', -10902779.0],
 ['應收帳款－關係人（增加）減少', -572949.0],
 ['其他應收款－關係人（增加）減少', 106673.0],
 ['存貨（增加）減少', -31475575.0],
 ['其他流動資產（增加）減少', -247466.0],
 ['其他金融資產（增加）減少', -5641723.0],
 ['其他營業資產（增加）減少', 139107.0],
 ['與營業活動相關之資產之淨變動合計', -47954908.0],
 ['應付帳款增加（減少）', 341340.0],
 ['應付帳款－關係人增加（減少）', -85053.0],
 ['負債準備增加（減少）', 0.0],
 ['其他流動負債增加（減少）', -26760684.0],
 ['淨確定福利負債增加(減少)', -62562.0],
 ['其他營業負債增加（減少）', -5629367.

In [78]:
# result = cursor.execute("SELECT * FROM t_2887_2013_4")
result = cursor.execute('SELECT * FROM t_final_data WHERE CO_ID="2887.TW"')
for row in result:
    print(row)
# print(result.fetchone())

('2887.TW', 2013, 1, 'Financial Services', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.031917320622141525, 0.037724123682682076, 0.00219121491589982, 0, 23.72154661906839, 1.7085839120753912, 1.7312255397213236, 0.07591027072368883)
('2887.TW', 2013, 2, 'Financial Services', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.03123840988121749, 0.036887879721339764, 0.002139297732696349, 0, 22.04264158515657, -0.17903645073975322, -0.15425416414924298, 0.13431022259662084)
('2887.TW', 2013, 3, 'Financial Services', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.02585686843036091, 0.030379365694977383, 0.0017674676519124461, 0, 22.643310004146908, -1.3654556790779298, -1.336506721000541, -0.04888454550206013)
('2887.TW', 2013, 4, 'Financial Services', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.018541669046846496, 0.02169982185113448, 0.001278672248081613, 0, 23.10627904444041, -2.9942925376692635, -2.9533268312632868, -0.006280939808762792)


### 2013~2018 檢查缺漏的欄位

In [41]:
print('rows missing:')

for year in range(2013, 2016):
    for season in range(1, 3):
        param = '2885', str(year), str(season)
        url = f'https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={param[0]}&SYEAR={param[1]}&SSEASON={param[2]}&REPORT_ID=C'
        
        print(f'    {year}, Q{season}')
        res = requests.get(url)
        res.encoding = 'big5'
        dfs = pd.read_html(StringIO(res.text))[1:4]

        # dfs[1]  # 資產負債表
        # dfs[2]  # 損益表
        # dfs[3]  # 現金流量表
        
        for i in range(3):
            dfs[i] = dfs[i].iloc[:, 0:2]
            dfs[i].columns = ['Code', 'Money']
            dfs[i].dropna(subset='Money', inplace = True)

        # store codes that exist in table, and compare with the truth value set
        tmpCodeSet = set()
        for i in range(0, 3):
            for key, val in Standard_Code_mapping[i].items():
                if len(dfs[i][dfs[i]['Code'] == key]) != 0:
                    tmpCodeSet.add(val)

        # print(CodeSet.difference(tmpCodeSet))     # 一般公司
        print(FinCompCodeSet.difference(tmpCodeSet))    # 金融公司
        

rows missing:
    2013, Q1
set()
    2013, Q2
set()
    2014, Q1
set()
    2014, Q2
set()
    2015, Q1
set()
    2015, Q2
set()


In [59]:
query = 'CREATE TABLE IF NOT EXISTS ' + 't_9999_TW' + ' (\
    Code TEXT PRIMARY KEY, \
    Title TEXT, \
    Money BIGINT \
    );'
print(query)
cursor.execute(query)
connection.commit()

CREATE TABLE IF NOT EXISTS t_9999_TW (    Code TEXT PRIMARY KEY,     Title TEXT,     Money BIGINT     );


## year before 2013 

In [34]:
url = 'https://mops.twse.com.tw/mops/web/ajax_t05st33'  # 資產負債表
url = 'https://mops.twse.com.tw/mops/web/ajax_t05st34'  # 損益表
url = 'https://mops.twse.com.tw/mops/web/ajax_t05st39'  # 現金表

In [94]:
import requests

# url = 'https://mops.twse.com.tw/mops/web/ajax_t05st33'  # 資產表
# url = 'https://mops.twse.com.tw/mops/web/ajax_t05st34'  # 損益表
url = 'https://mops.twse.com.tw/mops/web/ajax_t05st39'  # 現金表

payload = {
    'co_id': '1101',
    # 'co_id': '2330',
    # 'co_id': '2454',
    # 'co_id': '2885',
    # 'co_id': '2887',
    'year': '99',
    'season': '01',
    'firstin': '1',
    'check2858': 'Y',
    # 'checkbtn': '',
    # 'step': '1',
    # 'queryName': 'co_id',
    # 'TYPEK2': '',
    # 'inpuType': 'co_id',
    # 'off': '1',
    # 'year': 'Y',
    # 'keyword4': '',
    # 'isnew': 'false',
    # 'code1': '',
    # 'encodeURIComponent': '1',
    # 'TYPEK': 'sii'
    }

res = requests.post(url, data=payload)

# print(res.headers)
# print(res.cookies)
# print(res.text)
# print(res.status_code)
# print(res.encoding)
# print(res.json())
# print(res.url)

### Process: 資產表、損益表

In [92]:
dfs = pd.read_html(StringIO(res.text))

In [15]:
df = dfs[-1].iloc[4:, 0:2]   # drop first few rows

In [16]:
df.head(10)

,0


In [630]:
df.columns = ['Code', 'Money']

In [620]:
df.dropna(subset='Money', inplace = True)
df['Code'] = df['Code'].str.replace(' ','', regex=True) # remove space 

In [631]:
df['Money'] = pd.to_numeric(df['Money'])

#### 資產表

In [622]:
for key, val in Standard_Code_mapping[0].items():
    df.loc[df['Code'] == key , 'Code'] = val

In [623]:
# 一般公司

query_money = lambda key: df.loc[df['Code'] == key , 'Money'].values[0]
# 非流動資產合計 15XX
amount = query_money('1XXX') - query_money('11XX')
row = pd.DataFrame({'Code':['15XX'], 'Money': [amount]})
df = pd.concat([df, row], ignore_index=True)

# 非流動負債合計 25XX
amount = query_money('負債總計') - query_money('21XX')
row = pd.DataFrame({'Code':['25XX'], 'Money': [amount]})
df = pd.concat([df, row], ignore_index=True)


In [ ]:
# 股本 3100
if len(df[df['Code'] == '股本']) == 0:
    # 股本 = 普通股股本
    df.loc[df['Code'] == '普通股股本', 'Code'] = '3100'

#### 損益表

In [632]:
for key, val in Standard_Code_mapping[1].items():
    df.loc[df['Code'] == key , 'Code'] = val

In [73]:
df.loc[df['Code'] == '12' , 'Code']

Series([], Name: Code, dtype: object)

### Process: 現金表

In [107]:
dfs = pd.read_html(StringIO(res.text))

In [106]:
dfs[-1].values[0][0]

'台灣水泥股份有限公司及其子公司 合併現金流量表 民國九十九年及九十八年一月一日至三月三十一日 單位：新台幣仟元  九十九年第一季 九十八年第一季  -------------- -------------- 營業活動之現金流量： 合併總純益 $ 1,457,637 $ 486,224 調整項目 折舊及折耗 1,662,018 1,651,902  攤 銷 89,810 84,734  處分投資淨益 ( 1,567) ( 188) 採權益法認列之投資淨損(益) ( 9,707) 17,680  採權益法認列之被投資公司發放現金股利 19,534 -  處分資產淨益 ( 23,813) ( 1,113) 減損損失回升利益 ( 3,937) - 遞延所得稅 6,217 ( 71,929)  營業資產及負債之淨變動 公平價值變動列入損益之金融商品 58,149 ( 681,005)  應收票據 93,494 54,699 應收帳款 ( 775,926) ( 849,199) 應收關係人款項 ( 168,161) ( 194,063)  存 貨 ( 142,428) 1,551,380 預付款項 ( 54,870) ( 998,560) 其他流動資產 ( 259,784) 39,826  應付票據及帳款 1,591,625 1,385,949  應付關係人款項 73,636 27,793 應付所得稅 166,212 101,512 應付費用 ( 162,826) 310,902 其他流動負債 518,821 11,690  應計退休金負債 ( 27,410) 17,899  ------------ ------------ 營業活動之淨現金流入 4,106,544 2,946,133  ------------ ------------ 投資活動之現金流量： 取得持有至到期日金融資產價款 ( 2,322,154) -  處分以成本衡量之金融資產價款 3,711 -  採權益法之長期股權投資清算退回股款 4,758 -  購置資產 ( 1,705,409) ( 1,963,180)  質押銀行存款增加 ( 9,480,079) -  處分資產價款 24,509 4,234  其他資產減少（增加） ( 365,546) 300,028 ------------ -------

In [96]:
tmp = dfs[-1].values[0][0].replace('\u3000', '')    # remove Ideographic space
tmp = tmp.split(' ')
tmp

['台灣水泥股份有限公司及其子公司',
 '合併現金流量表',
 '民國九十九年及九十八年一月一日至三月三十一日',
 '單位：新台幣仟元',
 '',
 '九十九年第一季',
 '九十八年第一季',
 '',
 '--------------',
 '--------------',
 '營業活動之現金流量：',
 '合併總純益',
 '$',
 '1,457,637',
 '$',
 '486,224',
 '調整項目',
 '折舊及折耗',
 '1,662,018',
 '1,651,902',
 '',
 '攤',
 '銷',
 '89,810',
 '84,734',
 '',
 '處分投資淨益',
 '(',
 '1,567)',
 '(',
 '188)',
 '採權益法認列之投資淨損(益)',
 '(',
 '9,707)',
 '17,680',
 '',
 '採權益法認列之被投資公司發放現金股利',
 '19,534',
 '-',
 '',
 '處分資產淨益',
 '(',
 '23,813)',
 '(',
 '1,113)',
 '減損損失回升利益',
 '(',
 '3,937)',
 '-',
 '遞延所得稅',
 '6,217',
 '(',
 '71,929)',
 '',
 '營業資產及負債之淨變動',
 '公平價值變動列入損益之金融商品',
 '58,149',
 '(',
 '681,005)',
 '',
 '應收票據',
 '93,494',
 '54,699',
 '應收帳款',
 '(',
 '775,926)',
 '(',
 '849,199)',
 '應收關係人款項',
 '(',
 '168,161)',
 '(',
 '194,063)',
 '',
 '存',
 '貨',
 '(',
 '142,428)',
 '1,551,380',
 '預付款項',
 '(',
 '54,870)',
 '(',
 '998,560)',
 '其他流動資產',
 '(',
 '259,784)',
 '39,826',
 '',
 '應付票據及帳款',
 '1,591,625',
 '1,385,949',
 '',
 '應付關係人款項',
 '73,636',
 '27,793',
 '應付所得稅',
 '166,2

In [97]:
tmp_dict = {'Code':[], 'Money': []}

for i in range(len(tmp)):
    if tmp[i] in Standard_Code_mapping[2]:
        code = Standard_Code_mapping[2][tmp[i]]

        if len(tmp[i+1]) == 1:  # handle bad formatted data
            value = tmp[i+2]
        else:
            value = tmp[i+1]

        if '(' in value or ')' in value:
            value = '-' + value # negative

        value = value.replace(',', '')
        value = value.replace('(', '')
        value = value.replace(')', '')
        value = int(value) # to numeric value

        tmp_dict['Code'].append(code)
        tmp_dict['Money'].append(value)

        print(f'{code}: {value}')
        if code == 'B02700':    # handle edge case
            break


df = pd.DataFrame(tmp_dict) # build df

AAAA: 4106544
B02700: -1705409


In [99]:
df['Code'].to_list()

['AAAA', 'B02700']

In [100]:
df

,Code,Money
0,AAAA,4106544
1,B02700,-1705409


In [27]:
df.values.tolist()

[['AAAA', 3061775], ['B02700', -2697365]]

In [640]:
df[df['Code'] == '8200']

,Code,Money
35,8200,72455291.0


### Before 2013 檢查缺漏的欄位

In [ ]:
tmpCodeSet = set()

# 排除其他表
for key, val in Standard_Code_mapping[0].items():
    tmpCodeSet.add(val)
for key, val in Standard_Code_mapping[2].items():
    tmpCodeSet.add(val)

# 檢查的目標
for key, val in Standard_Code_mapping[1].items():
    if len(df[df['Code'] == key]) != 0:
        tmpCodeSet.add(val)

print(CodeSet.difference(tmpCodeSet))     # 一般公司
# print(FinCompCodeSet.difference(tmpCodeSet))    # 金融公司

## Test snippets

In [25]:
import tejapi
tejapi.ApiConfig.api_key = "rdgK8i4xzvNt9Wis2kKEo9qyyon5iG"

In [27]:
tb_info = tejapi.get('TRAIL/AIND')

In [31]:
tb_info

,coid,mdate,mkt,elist_day1,tseid,ind,ind1,emm,ind_ban,isin,...,tejinm1_e,tejico2,tejinm2_c,tejinm2_e,tejico3,tejinm3_c,tejinm3_e,tejico4,tejinm4_c,tejinm4_e
None,,,,,,,,,,,,,,,,,,,,,
0,000116,2022-01-01 00:00:00+00:00,,1996-06-28 00:00:00+00:00,000116,30,17,12,ID 03098908,TW0006014004,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
1,000930,2022-01-01 00:00:00+00:00,,1995-12-27 00:00:00+00:00,000930,30,17,12,ID 22955774,TW0006009004,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
2,000960,2022-01-01 00:00:00+00:00,,1995-12-12 00:00:00+00:00,000960,30,17,12,ID 22957301,TW0006007008,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
3,0009A0,2022-01-01 00:00:00+00:00,,1994-12-06 00:00:00+00:00,0009A0,30,17,12,ID 23113343,TW0006001001,...,Securities,M30A,證券,Securities Firm,M30A,證券,Securities Firm,M2800,金融業,Financial
4,1101,2022-01-01 00:00:00+00:00,TSE,1962-02-09 00:00:00+00:00,1101,11,01,12,ID 11913502,TW0001101004,...,Cement,M11A,水泥製造,Cement Manufacturing,M11A,水泥製造,Cement Manufacturing,M1100,水泥工業,Cement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,9962,2022-01-01 00:00:00+00:00,OTC,2006-07-10 00:00:00+00:00,9962,20,10,12,ID 96862363,TW0009962001,...,Iron and Steel,M20A,金屬基本,Metal,M20A6,不鏽鋼,Stainless Steel,M2000,鋼鐵工業,Iron & Steel
3131,9965,2022-01-01 00:00:00+00:00,,2006-12-29 00:00:00+00:00,9965,26,15,12,ID 22765839,TW0009965004,...,Shipping and Trans.,M26C,貨運倉儲業,Freight Warehouse,M26C,貨運倉儲業,Freight Warehouse,M2600,航運業,Shipping & Trans.
3132,L2301,2022-01-01 00:00:00+00:00,,1983-01-26 00:00:00+00:00,L2301,23,13,12,ID 04631182,TW0002301009,...,Electronics,M23D,電子零組件,Electron Component,M23D3,電源供應器,Power Supply Unit,,,


In [43]:
col = tejapi.get('TRAIL/TAIACC')


In [45]:
col.iloc[100:]

,id,code,acseq,cname,ename,unit,cgrp,egrp
None,,,,,,,,
100,TAIM1A,2480,71,其他權益,Total other equity interest,T,,
101,TAIM1A,2801,74,共同控制下前手權益,Interests in the former associate or jointly c...,T,,
102,TAIM1A,2802,75,合併前非屬共同控制股權,Non-controlling interest in pre-acquisition sh...,T,,
103,TAIM1A,2900,76,非控制權益,Non-controlling interests,T,,
104,TAIM1A,3100,79,營業收入淨額,Total operating revenue,T,,
...,...,...,...,...,...,...,...,...
271,TAIM1A,R699,230,淨負債,Net DEBT,T,,
272,TAIM1A,R69B,251,自由現金流量(D),Free Cash Flow (D),T,,
273,TAIM1A,R834,256,每人營收,Revenue Per Employee,T,,


In [36]:
data = tejapi.get('TRAIL/TAIM1AQ')

/Users/hsiao/opt/anaconda3/lib/python3.9/site-packages/tejapi/get.py:43: UserWarning: 如需要更多資料，請設定參數 paginate=True，詳細訊息: %s 
  warnings.warn(Message.WARN_PAGE_LIMIT_EXCEEDED, UserWarning)


In [42]:
data.iloc[8000:]

,coid,mdate,acc_code,acc_value
None,,,,
8000,000144,2021-12-01 00:00:00+00:00,R502,134.76
8001,000144,2021-12-01 00:00:00+00:00,R503,0.31
8002,000144,2021-12-01 00:00:00+00:00,R504,270.74
8003,000144,2021-12-01 00:00:00+00:00,R505,73.03
8004,000144,2021-12-01 00:00:00+00:00,R506,26.97
...,...,...,...,...
9995,000159,2021-06-01 00:00:00+00:00,R501,139.74
9996,000159,2021-06-01 00:00:00+00:00,R502,139.26
9997,000159,2021-06-01 00:00:00+00:00,R503,0.04


In [2]:
import requests
import pandas as pd
import csv

In [26]:
co_id = "2330.TW"
Year = 2022 # 2013 Q1 ~ 2022 Q1
Quarter = 1 # 1, 2, 3, 4
url = f'http://140.113.207.34:8000/company_data/{co_id}/final_data?Year={Year}&Quarter={Quarter}'

response = requests.get(url)
entry = response.json()

print(entry)


{'data_exist': True, 'CO_ID': '2330.TW', 'Year': 2022, 'Quarter': 1, 'Sector': 'Technology', 'ratio1': 2.092970251899799, 'ratio2': 0.2202436395597397, 'ratio3': 0.28245186667715966, 'ratio4': 0.5563359574784078, 'ratio5': 0.45571393404619576, 'ratio6': 0.4619076368266213, 'ratio7': 0.4131202226666917, 'ratio8': 0.12299415002942937, 'ratio9': 1.0887174791606913, 'ratio10': 2.3071789805940837, 'ratio11': 158.20185736349544, 'ratio12': 0.0873900945900907, 'ratio13': 0.08836530636139223, 'ratio14': 0.05081137064685834, 'ratio15': 0.25736673643700647, 'ratio16': 89.52219527650786, 'ratio17': 14.35188280166402, 'ratio18': 24.46051573615258, 'Stock_Return': 0.004114905451062561}
